In [2]:
setwd(paste0(Sys.getenv('ROOT'), '/R'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/R"

In [14]:
source('main.R')

data <- kaggle.house$loadData()

df.combined <- kaggle.house$getCombinedDataset(data$train, data$test) %>% (kaggle.house$na$fixAll)

df.training.raw <- 
    df.combined %>% 
    filter(dataSource == "train") %>% 
    mutate(sale_price_log = log(SalePrice)) %>%
    select(-dataSource, -SalePrice)
    
df.testing.raw  <- 
    df.combined %>% 
    filter(dataSource == "test") %>% 
    select(-dataSource, -SalePrice)

In [15]:
df.training.numeric <- 
    df.training.raw %>%
    select(-dplyr::one_of(kaggle.house$trans$type1TransContainer %>% names)) %>%
    select(-dplyr::one_of(kaggle.house$trans$type2TransContainer %>% names))

# Should be only numeric columns
stopifnot(0 == df.training.numeric %>% purrr::map(function (col) { !is.numeric(col) }) %>% unlist %>% sum)

df.training.numeric %>% colnames %>% sort

[1] "BedroomAbvGr"   "BsmtFinSF1"     "BsmtFinSF2"     "BsmtFullBath"  
 [5] "BsmtHalfBath"   "BsmtUnfSF"      "EnclosedPorch"  "Fireplaces"    
 [9] "FullBath"       "GarageArea"     "GarageCars"     "GarageYrBlt"   
[13] "GrLivArea"      "HalfBath"       "KitchenAbvGr"   "LotArea"       
[17] "LotFrontage"    "LowQualFinSF"   "MasVnrArea"     "MiscVal"       
[21] "MoSold"         "OpenPorchSF"    "OverallCond"    "OverallQual"   
[25] "PoolArea"       "sale_price_log" "ScreenPorch"    "TotalBsmtSF"   
[29] "TotRmsAbvGrd"   "WoodDeckSF"     "X1stFlrSF"      "X2ndFlrSF"     
[33] "X3SsnPorch"     "YearBuilt"      "YearRemodAdd"   "YrSold"

In [91]:
x <- 
    df.training.numeric %>% 
    gather(name, value, -sale_price_log) %>%
    group_by(name) %>%
    nest %>% 
    mutate(
        mod = map(data, ~lm(sale_price_log ~ value, .)),
        glance = map(mod, broom::glance),
        r.squared = map_dbl(glance, 'r.squared'),
        
        mod.log = map(data, ~lm(sale_price_log ~ log(value + 1), .)),
        glance.log = map(mod.log, broom::glance),
        r.squared.log = map_dbl(glance.log, 'r.squared'),
        
        mod.sqrt = map(data, ~lm(sale_price_log ~ sqrt(value ), .)),
        glance.sqrt = map(mod.sqrt, broom::glance),
        r.squared.sqrt = map_dbl(glance.sqrt, 'r.squared')
    )

x %>% 
    arrange(desc(r.squared)) %>% 
    select(name, r.squared, r.squared.log, r.squared.sqrt) %>%
     mutate(
         log = r.squared.log / r.squared, 
         sqrt = r.squared.sqrt / r.squared
     ) %>%
     select(name, log, sqrt) %>%
     gather(fun, value, -name) %>%
     arrange(name, desc(value)) %>%
     group_by(name) %>%
     filter(row_number() == 1) %>%
     arrange(desc(value)) %>%
     filter(value > 1.2) %>%
     select(`Attribute Name` = name, `Transformation` = fun, `Transformation R.Squared Gain` = value)

Attribute Name,Transformation,Transformation R.Squared Gain
BsmtFinSF2,log,39.472132
MiscVal,log,11.217093
LotArea,log,2.415495
LowQualFinSF,log,2.062000
OpenPorchSF,log,2.052408
EnclosedPorch,log,1.933655
